In [97]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import metrics
from tensorflow.keras.layers import Dense
from tensorflow.keras import activations
from tensorflow.keras import callbacks
from tensorflow.keras import models

Work rate and Position are two variables which can be related to a footballer's value, so they are selected for one hot encoding. Columns in the remove_cols list are those which are not needed for determining a player's value.

In [98]:
cols = pd.read_csv("data.csv", nrows = 0).columns.tolist()
remove_cols = [0, 1, 2, 4, 5, 6, 9, 10, 13, 19, 20, 22, 23, 24, 25, 26, 27, 88]
cats = ["Work Rate", "Position"]
for i in range(28, 54):
    remove_cols.append(i)

In [99]:
def currency_to_number(np_array):
    np_array = np_array.str.replace("€", "")
    np_array = np_array.replace({'K': '*1e3', 'M': '*1e6'}, 
                                regex=True).map(pd.eval).astype(float)
    return np_array

In [100]:
def create_df(file_name):
    _df = pd.read_csv(file_name, delim_whitespace=False, names=cols, skiprows=1)
    names = _df["Name"]
    _df.drop(_df.columns[remove_cols],axis=1,inplace=True)
    _df["Preferred Foot"].replace("Left", 1, inplace=True)
    _df["Preferred Foot"].replace("Right", 0, inplace=True)
    _df = pd.get_dummies(_df, prefix=cats, columns=cats)
    _df["Value"] = currency_to_number(_df["Value"])
    _df["Wage"] = currency_to_number(_df["Wage"])
    return _df, names

df, _ = create_df("data.csv")
mins = {}
maxs = {}
for a in df.columns:
    if a not in cats:
        mins[a] = df[a].min()
        maxs[a] = df[a].max()

def scale_data(_df):
    for a in _df.columns:
        if a not in cats:
            _df[a] = (_df[a] - mins[a]) / (maxs[a] - mins[a])
    return _df

df = scale_data(df)
corr_matrix = df.corr()
drop_cols = []
for i in range(0, len(corr_matrix["Value"])):
    print(corr_matrix.columns[i] + ": " + str(abs(corr_matrix["Value"][i])))
    if 0.5 < abs(corr_matrix["Value"][i]) < 1:
        drop_cols.append(i)
    if "GK" in df.columns[i]:
        df[df.columns[i]] = df[df.columns[i]] * df["Position_GK"]
df = df.drop(df.columns[drop_cols], axis=1).fillna(0)
df[df < 0] = 0
df[df > 1] = 1

df

Age: 0.07502220683279813
Overall: 0.6270852347261893
Potential: 0.576218252981702
Value: 1.0
Wage: 0.8584417232333243
Preferred Foot: 0.01681067025006109
International Reputation: 0.6531762672380879
Weak Foot: 0.1652214685620562
Skill Moves: 0.3156514601779245
Crossing: 0.24978486077778106
Finishing: 0.25666485675299805
HeadingAccuracy: 0.18443722458647696
ShortPassing: 0.3276231298507388
Volleys: 0.2882432225395833
Dribbling: 0.27200389086140103
Curve: 0.28723476490784483
FKAccuracy: 0.26577334857148355
LongPassing: 0.3021749622454012
BallControl: 0.3088179609587565
Acceleration: 0.17060266381477923
SprintSpeed: 0.17267650125224127
Agility: 0.19425920178546716
Reactions: 0.5405519335346208
Balance: 0.1137255206160286
ShotPower: 0.28136036569665884
Jumping: 0.12451502208513653
Stamina: 0.21072086055193207
Strength: 0.12892396479826268
LongShots: 0.2797296720120012
Aggression: 0.1846389944689045
Interceptions: 0.14066644566198275
Positioning: 0.25939063707220156
Vision: 0.35763929755246

,Age,Value,Preferred Foot,Weak Foot,Skill Moves,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Work Rate_High/ High,Work Rate_High/ Low,Work Rate_High/ Medium,Work Rate_Low/ High,Work Rate_Low/ Low,Work Rate_Low/ Medium,Work Rate_Medium/ High,Work Rate_Medium/ Low,Work Rate_Medium/ Medium,Position_CAM,Position_CB,Position_CDM,Position_CF,Position_CM,Position_GK,Position_LAM,Position_LB,Position_LCB,Position_LCM,Position_LDM,Position_LF,Position_LM,Position_LS,Position_LW,Position_LWB,Position_RAM,Position_RB,Position_RCB,Position_RCM,Position_RDM,Position_RF,Position_RM,Position_RS,Position_RW,Position_RWB,Position_ST
0,0.517241,0.932489,1.0,0.75,0.75,0.897727,1.000000,0.733333,0.965116,0.953488,1.000000,0.988636,1.000000,0.928571,1.000000,0.929412,0.880952,0.939024,0.9875,0.892473,0.6625,0.714286,0.5250,1.000000,0.440476,0.213483,0.989247,1.000000,0.804598,1.000000,0.329670,0.285714,0.261364,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.586207,0.649789,0.0,0.75,1.00,0.897727,0.989247,0.944444,0.860465,0.965116,0.903226,0.852273,0.802198,0.809524,0.978022,0.905882,0.940476,0.890244,0.6750,1.000000,1.0000,0.904762,0.7750,0.989011,0.619048,0.292135,1.000000,0.857143,0.919540,0.989247,0.274725,0.318681,0.227273,0.0,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.344828,1.000000,0.0,1.00,1.00,0.840909,0.913978,0.644444,0.895349,0.930233,0.989247,0.931818,0.923077,0.821429,0.989011,0.964706,0.928571,1.000000,0.8500,0.838710,0.5750,0.821429,0.4000,0.868132,0.535714,0.370787,0.935484,0.916667,0.873563,0.978495,0.263736,0.241758,0.340909,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.379310,0.607595,0.0,0.50,0.00,0.136364,0.118280,0.188889,0.500000,0.104651,0.150538,0.170455,0.175824,0.500000,0.406593,0.529412,0.547619,0.560976,0.3375,0.311828,0.6500,0.369048,0.5875,0.098901,0.321429,0.303371,0.107527,0.690476,0.402299,0.698925,0.131868,0.208791,0.113636,1.0,0.923077,0.955556,0.977528,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.379310,0.860759,0.0,1.00,0.75,1.000000,0.860215,0.566667,0.988372,0.906977,0.881720,0.897727,0.879121,0.976190,0.945055,0.776471,0.761905,0.792683,0.7625,0.956989,0.6000,0.928571,0.7250,0.967033,0.773810,0.651685,0.913978,1.000000,0.850575,0.913978,0.714286,0.615385,0.545455,0.0,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,0.103448,0.000506,0.0,0.25,0.25,0.329545,0.387097,0.400000,0.488372,0.244186,0.408602,0.272727,0.340659,0.428571,0.417582,0.494118,0.535714,0.560976,0.7500,0.440860,0.5000,0.333333,0.3750,0.384615,0.416667,0.483146,0.397849,0.500000,0.436782,0.451613,0.406593,0.505495,0.500000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18203,0.103448

In [103]:
y = df["Value"]
X = df.drop(columns=["Value"])
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3)

weight_neuron = []

train_len = len(xtrain)
test_len = len(xtest)
y_len = len(ytrain)

class CustomCallback(callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        temp = self.model.layers[2].get_weights()[0].tolist()
        weight_neuron.append(temp)

model = models.Sequential()
model.add(Dense(180,activation="sigmoid",input_dim=73))
model.add(Dense(100,activation="sigmoid"))
model.add(Dense(1,activation="sigmoid"))

model.compile(optimizer="adam",
            loss="mean_squared_error")

weight_neuron = []

history_train = model.fit(xtrain, ytrain,
                batch_size=32, epochs=100,
                validation_data=(xtest, ytest),
                callbacks=[CustomCallback()],
                )


Epoch 1/100
399/399 [==============================] - 1s 3ms/step - loss: 0.0046 - val_loss: 0.0024
Epoch 2/100
399/399 [==============================] - 1s 2ms/step - loss: 0.0021 - val_loss: 0.0024
Epoch 3/100
399/399 [==============================] - 1s 2ms/step - loss: 0.0021 - val_loss: 0.0023
Epoch 4/100
399/399 [==============================] - 1s 2ms/step - loss: 0.0020 - val_loss: 0.0022
Epoch 5/100
399/399 [==============================] - 1s 2ms/step - loss: 0.0019 - val_loss: 0.0020
Epoch 6/100
399/399 [==============================] - 1s 2ms/step - loss: 0.0018 - val_loss: 0.0018
Epoch 7/100
399/399 [==============================] - 1s 2ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 8/100
399/399 [==============================] - 1s 2ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 9/100
399/399 [==============================] - 1s 2ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 10/100
399/399 [==============================] - 1s 2ms/step - loss: 9.5681e-04 - va

In [102]:
new_df, names = create_df("test_data.csv")
new_df = scale_data(new_df)
for col in new_df.columns:
    if col not in X.columns:
        new_df.drop(columns=[col], inplace=True)
for i in range(0, len(X.columns)):
    if X.columns[i] not in new_df.columns:
        index = new_df.index
        number_of_rows = len(index)
        new_df.insert(i, X.columns[i], np.zeros(number_of_rows))

predictions = model.predict(new_df)
for i in range(0, len(predictions)):
    val = (predictions[i] * (maxs["Value"] - mins["Value"])) + mins["Value"]
    prefix = ""
    if val > 1000000:
        prefix = "M"
        val = val / 1000000
    elif val > 1000:
        prefix = "K"
        val = val / 1000
    print(names[i] + " is worth €" + str(val[0].round(2)) + prefix)

A is worth €51.34M
B is worth €68.11M
C is worth €74.19M
D is worth €9.5M
E is worth €90.77M
F is worth €3.13M
G is worth €58.63M
H is worth €90.77M
I is worth €5.27K
